In [1]:
from astroquery.vizier import Vizier


In [2]:
Vizier.ROW_LIMIT = -1
cat, = Vizier.get_catalogs('J/ApJS/199/26/table3')
len(cat)

44599

In [3]:
import sys
sys.path.append('$POCS/PocsAlerter')

from email_parser import ParseEmail

2017-01-16 11:09:07 bernie.localdomain panoptes[21887] INFO Loading state table: simple_state_table
2017-01-16 11:09:07 bernie.localdomain panoptes[21887] INFO ********************************************************************************
2017-01-16 11:09:07 bernie.localdomain panoptes[21887] INFO Initializing PANOPTES unit
2017-01-16 11:09:07 bernie.localdomain panoptes[21887] INFO Welcome Generic PANOPTES Unit!
2017-01-16 11:09:07 bernie.localdomain panoptes[21887] INFO 	 observatory
2017-01-16 11:09:07 bernie.localdomain panoptes[21887] INFO 	Initializing observatory
2017-01-16 11:09:07 bernie.localdomain panoptes[21887] INFO 		 Setting up location
2017-01-16 11:09:07 bernie.localdomain panoptes[21887] INFO 		 Setting up mount
2017-01-16 11:09:07 bernie.localdomain panoptes[21887] INFO 		Using simulator mount
2017-01-16 11:09:07 bernie.localdomain panoptes[21887] INFO 		 Setting up cameras
2017-01-16 11:09:07 bernie.localdomain panoptes[21887] INFO 		 Setting up scheduler
2017-01-

In [4]:
mail = ParseEmail('imap.gmail.com', 'bernie.telalovic@gmail.com', 'hdzlezqobwooqdqt', test=True)

In [5]:
read, text = mail.get_email('LVC_TEST')

Mon, 09 Jan 2017 11:18:42


In [ ]:
message = mail.read_email(text)

In [ ]:
message['SKYMAP_URL'] = 'https://dcc.ligo.org/P1500071/public/10458_bayestar.fits.gz'
mail.parse_event(message)

2017-01-16 11:09:26 bernie.localdomain panoptes[21887] INFO Loading state table: simple_state_table
2017-01-16 11:09:26 bernie.localdomain panoptes[21887] INFO ********************************************************************************
2017-01-16 11:09:26 bernie.localdomain panoptes[21887] INFO Initializing PANOPTES unit
2017-01-16 11:09:26 bernie.localdomain panoptes[21887] INFO Welcome Generic PANOPTES Unit!
2017-01-16 11:09:26 bernie.localdomain panoptes[21887] INFO 	 observatory
2017-01-16 11:09:26 bernie.localdomain panoptes[21887] INFO 	Initializing observatory
2017-01-16 11:09:26 bernie.localdomain panoptes[21887] INFO 		 Setting up location
2017-01-16 11:09:26 bernie.localdomain panoptes[21887] INFO 		 Setting up mount
2017-01-16 11:09:26 bernie.localdomain panoptes[21887] INFO 		Using simulator mount
2017-01-16 11:09:26 bernie.localdomain panoptes[21887] INFO 		 Setting up cameras
2017-01-16 11:09:26 bernie.localdomain panoptes[21887] INFO 		 Setting up scheduler
2017-01-

In [ ]:
tiles = mail.checked_targets
print(len(tiles))

In [ ]:
print(message)

In [ ]:
print(message['MAX_DIST'].split('[')[0])

In [ ]:
from astropy.time import Time

In [ ]:
import numpy as np

In [ ]:
arr = np.array([1.0, 1.0, 1.0, 1.0, 1.0, 1.0])
print(arr >= np.nanpercentile(arr, 98))

In [ ]:
def get_redshift(cat):
    
    z = (u.Quantity(cat['cz']) / c.c).to(u.dimensionless_unscaled)
    MK = cat['Ktmag'] - cosmo.distmod(z)
    
    return z

In [ ]:
import healpy as hp
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import norm

from matplotlib import pyplot as plt
from astropy.cosmology import WMAP9 as cosmo
from astropy.table import Column
import astropy.units as u
import astropy.constants as c
from matplotlib import colors
import math as m
import numpy as np

from astropy.coordinates import EarthLocation
from pytz import timezone
from astroplan import Observer
from astropy.time import Time
import datetime

from astroquery.skyview import SkyView
from astropy.coordinates import SkyCoord
from astroquery.irsa_dust import IrsaDust
from astropy.io import fits as pyfits
import glob

In [ ]:
from astropy.utils.data import download_file

url = ('https://dcc.ligo.org/P1500071/public/10458_bayestar.fits.gz')
filename = download_file(url, cache=True)

In [ ]:
z = get_redshift(cat)

prob, distmu, distsigma, distnorm = hp.read_map(filename, field=range(4))

npix = len(prob)
print(npix)
nside = hp.npix2nside(npix)
pixarea = hp.nside2pixarea(nside)
pixarea

cat_now = cat
print(type(cat))

r = cosmo.luminosity_distance(z).to('Mpc').value
theta = 0.5*np.pi - cat_now['_DEJ2000'].to('rad').value
phi = cat_now['_RAJ2000'].to('rad').value
ipix = hp.ang2pix(nside, theta, phi)
dp_dV = prob[ipix]*distnorm[ipix]*norm(distmu[ipix], distsigma[ipix]).pdf(r) / pixarea
print(max(dp_dV))

cands = cat_now[(dp_dV >= np.nanpercentile(dp_dV,95)) & (r <= 50.0)]

cands_hist = cat_now[(dp_dV >= np.nanpercentile(dp_dV,95)) & (r <= 102.71)]

print(len(cands_hist))
len_cands = len(cands_hist)

org_ra = cands_hist['_RAJ2000']
org_dec = cands_hist['_DEJ2000']
weights = []

for ra_ind in range(len(cands_hist['_RAJ2000'])):
    weight = abs(1/np.cos(cands_hist['_DEJ2000'][ra_ind]))
    if weight > 1.0:
        weight = 1.0
    weights.append(weight)


In [ ]:
dp_dV/max(dp_dV)

In [ ]:
score = []
ras = []
decs = []
for tile in tiles:
    scr = tile['properties']['score']
    #for gal in tile['galaxies']:
    if scr > 10e-50:
        scr = len(tile['gal_indexes'])
        coords = SkyCoord(tile['properties']['coords_num'][0], tile['properties']['coords_num'][1], unit='deg', frame='fk5')
        dec = coords.dec.deg
        decs.append(coords.dec.deg)
        ras.append(coords.ra.deg)
        score.append(scr)

In [ ]:
%matplotlib inline

In [ ]:
plt.figure(figsize=(20,10))
from matplotlib import cm
afm_cmap = cm.summer
afm_cmap.set_under("w") # sets background to white
plt.hist2d(org_ra, org_dec, bins = [144, 90],range=np.array([(0,360), (-90,90)]), cmap = afm_cmap, cmax=80)
cbar = plt.colorbar()
cbar.ax.set_ylabel('Number of Galaxies per tile')

plt.scatter(ras, decs, marker='s', color='b', s=40, alpha=.5)
plt.axhline(y=-31, color='r', linestyle='-')
plt.axhline(y=20, color='r', linestyle='--')
plt.axhline(y=-81, color='r', linestyle='--')

plt.xlabel('RA (deg)', fontsize = 15)
plt.ylabel('DEC (deg)', fontsize=15)
plt.title('Tiling Observable Region \n', fontsize=20)
plt.xlim([0,360])
plt.ylim([-90,90])

In [ ]:
import matplotlib

theta = 0.5*np.pi - np.radians(decs)
phi = np.radians(ras)

thing = prob/max(prob)


maxim = np.nanpercentile(thing, 95.0)
winter = cm.cubehelix_r
winter.set_under("w")
hp.mollview(thing, coord='C',
            title='Probability Map With Transposed Tiles Selected\n', cmap = winter, min = 0.0, max = 1.0)
            
#norm = matplotlib.colors.LogNorm())

s = hp.projscatter(theta, phi, c=score, marker='o', s=20, alpha=1.0, linewidth = 0.0)
s.cmap = cm.plasma
hp.graticule()
winter = cm.cool
cb = plt.colorbar(s, orientation = 'horizontal', shrink = 0.5)
cb.set_label('Number of Galaxies In Tile')

